# Build a RAG Example with TiDB, DSPy, Meta Llama 3, Amazon Bedrock and Boto3

## Introduction

In this notebook we will show you how to use TiDB and Boto3 SDK to build a RAG with Meta Llama 3 and Amazon Bedrock. [TiDB](https://tidb.cloud/?utm_source=github&utm_medium=community&utm_campaign=video_aws_example_generativeai_cm_0624) is an open-source distributed SQL database that supports Hybrid Transactional and Analytical Processing (HTAP) workloads. It is MySQL compatible and features horizontal scalability, strong consistency, and high availability. You can deploy TiDB in a self-hosted environment or in the cloud.

### Use case

To demonstrate the vector search capability of TiDB Serverless, and the text generation capability of Meta Llama 3, let's take the use case of build a RAG Q&A bot.

### Persona

You are a TiDB user, you want to build a searching engine about TiDB. So you can build a RAG Q&A bot to achieve it.

### Implementation

To fulfill this use case, in this notebook we will show how to build a RAG application. Save the information to TiDB Serverless, use vector search feature in TiDB Serverless to get information. And then, we will use those information to generate the answer via Meta Llama 3. We will use the TiDB, and the Meta Llama 3 model through the Amazon Bedrock API with Boto3 SDK.

### Python 3.10

⚠  For this lab we need to run the notebook based on a Python 3.10 runtime. ⚠

### Setting

Before you run this Jupyter Notebook, please set the environment variables:

- TIDB_HOST
- TIDB_PORT
- TIDB_USER
- TIDB_PASSWORD
- TIDB_DB_NAME


> **Warning:**
>
> Aware that this notebook will drop some tables and recreate them, please use a new TiDB Serverless cluster.

## Installation

To run this notebook you would need to install dependencies - SQLAlchemy, tidb-vector, DSPy, Langchain Community, PyMySQL, pydantic, boto3, pyvis.

In [ ]:
%%capture
%pip install PyMySQL==1.1.0 --force-reinstall --quiet
%pip install SQLAlchemy==2.0.30 --force-reinstall --quiet
%pip install tidb-vector==0.0.9 --force-reinstall --quiet
%pip install pydantic==2.7.1 --force-reinstall --quiet
%pip install pydantic_core==2.18.2 --force-reinstall --quiet
%pip install boto3 --force-reinstall --quiet

## Kernel Restart

Restart the kernel with the updated packages that are installed through the dependencies above

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

## Setup

Import the necessary libraries

In [ ]:
import json
import os
import boto3

from sqlalchemy import (
    Column,
    Integer,
    Text,
    URL,
    create_engine,
)
from sqlalchemy.orm import Session, declarative_base
from tidb_vector.sqlalchemy import VectorType

## Initialization

Connect to a TiDB Cloud Cluster and Initiate Bedrock Runtime Client

In [ ]:
def get_db_url():
    return URL(
        drivername="mysql+pymysql",
        username=os.environ["TIDB_USER"],
        password=os.environ["TIDB_PASSWORD"],
        host=os.environ['TIDB_HOST'],
        port=int(os.environ["TIDB_PORT"]),
        database=os.environ["TIDB_DB_NAME"],
        query={"ssl_verify_cert": True, "ssl_verify_identity": True},
    )

engine = create_engine(get_db_url(), pool_recycle=300)
bedrock_runtime = boto3.client('bedrock-runtime')

## Model invocation

Invoke the Amazon Titan Text Embeddings V2 model using bedrock runtime client

Amazon Bedrock runtime client provides you with an API `invoke_model` which accepts the following:
- `modelId`: This is the model ARN for the foundation model available in Amazon Bedrock
- `accept`: The type of input request
- `contentType`: The content type of the output
- `body`: A json string payload consisting of the prompt and the configurations

In [ ]:
embedding_model_name = "amazon.titan-embed-text-v2:0"
dim_of_embedding_model = 512
llm_name = "meta.llama3-70b-instruct-v1:0"

def embedding(content):
    payload = {
        "modelId": embedding_model_name,
        "contentType": "application/json",
        "accept": "*/*",
        "body": {
            "inputText": content,
            "dimensions": dim_of_embedding_model,
            "normalize": True,
        }
    }

    # Convert the payload to bytes
    body_bytes = json.dumps(payload['body']).encode('utf-8')

    # Invoke the model
    response = bedrock_runtime.invoke_model(
        body=body_bytes,
        contentType=payload['contentType'],
        accept=payload['accept'],
        modelId=payload['modelId']
    )

    result_body = json.loads(response.get("body").read())

    return result_body.get("embedding")

def generate_result(query: str, info_str: str):
    prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
ONLY use the content below to generate an answer:
{info_str}<|eot_id|>

<|start_header_id|>user<|end_header_id|>

Please carefully think about the question: {query}<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
"""

    payload = {
        "modelId": llm_name,
        "contentType": "application/json",
        "accept": "application/json",
        "body": {
            "prompt": prompt,
            "temperature": 0
        }
    }

    # Convert the payload to bytes
    body_bytes = json.dumps(payload['body']).encode('utf-8')

    # Invoke the model
    response = bedrock_runtime.invoke_model(
        body=body_bytes,
        contentType=payload['contentType'],
        accept=payload['accept'],
        modelId=payload['modelId']
    )

    result_body = json.loads(response.get("body").read())
    completion = result_body["generation"]
    return completion

## TiDB Table and Vector Index

Create table and its vector index in TiDB Serverless to store text and vector.

In [ ]:
Base = declarative_base()
class Entity(Base):
    __tablename__ = "entity"

    id = Column(Integer, primary_key=True)
    content = Column(Text)
    content_vec = Column(
        VectorType(dim=dim_of_embedding_model),
        comment="hnsw(distance=l2)"
    )

Base.metadata.create_all(engine)

## Save Vector to TiDB Serverless

Save 5 records with embedding vector to TiDB Serverless.

In [ ]:
tidb_content = 'TiDB is an open-source distributed SQL database that supports Hybrid Transactional and Analytical Processing (HTAP) workloads.'
tikv_content = 'TiKV is an open-source, distributed, and transactional key-value database. Unlike other traditional NoSQL systems.'
tiflash_content = 'TiFlash is the key component that makes TiDB essentially an Hybrid Transactional/Analytical Processing (HTAP) database. As a columnar storage extension of TiKV, TiFlash provides both good isolation level and strong consistency guarantee.'
pd_content = 'The Placement Driver (PD) server is the metadata managing component of the entire cluster.'
tidb_cloud_content = 'TiDB Cloud is a fully-managed Database-as-a-Service (DBaaS) that brings TiDB, an open-source Hybrid Transactional and Analytical Processing (HTAP) database, to your cloud. TiDB Cloud offers an easy way to deploy and manage databases to let you focus on your applications, not the complexities of the databases.'

with Session(engine) as session:
    session.add(Entity(content = tidb_content, content_vec = embedding(tidb_content)))
    session.add(Entity(content = tikv_content, content_vec = embedding(tikv_content)))
    session.add(Entity(content = tiflash_content, content_vec = embedding(tiflash_content)))
    session.add(Entity(content = pd_content, content_vec = embedding(pd_content)))
    session.add(Entity(content = tidb_cloud_content, content_vec = embedding(tidb_cloud_content)))
    session.commit()

### Ask Question

In [ ]:
question = "What is the relationship between TiKV and TiFlash?" # @param {type:"string"}

### Find Information by Vector Search

In this case, we will get the nearest 3 contents, by using embedding vector which the feature offered by TiDB Serverless.


In [ ]:
question_embedding = embedding(question)
with Session(engine) as session:
    info_list = session.query(Entity) \
        .order_by(Entity.content_vec.cosine_distance(question_embedding)) \
        .limit(3).all()

### Generate Answer

Once we got the entities and relationships, we can generate the answer via the Meta Llama 3.

In [ ]:
info_str = '\n'.join(map(lambda info: info.content, info_list))
result = generate_result(question, info_str)
result